In [7]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit import execute
from qiskit import IBMQ
import math
import operator
from qiskit import Aer

def createInputState(qc, reg, n, pie):
    """
    Apply one Hadamard gate to the nth qubit of the quantum register               
    reg, and then apply repeated phase rotations with parameters  
    being pi divided by increasing powers of two.
    """
    qc.h(reg[n])    
    for i in range(0, n):
        qc.cu1(pie/float(2**(i+1)), reg[n-(i+1)], reg[n])
        
def evolveQFTState(qc, reg_a, reg_b, n, pie, factor):
    """  
    Evolves the state |F(ψ(reg_a))> to |F(ψ(reg_a+reg_b))> using the   
    quantum Fourier transform conditioned on the qubits of the 
    reg_b. Apply repeated phase rotations with parameters being pi 
    divided by increasing powers of two.
    """
    l = len(reg_b)
    for i in range(0, n + 1):
        if (n - i) > l - 1:
            pass
        else:
            qc.cu1(factor * pie / float(2**(i)), reg_b[n - i], reg_a[n])
            
def inverseQFT(qc, reg, n, pie):
    """
    Performs the inverse quantum Fourier transform on a register 
    reg. Apply repeated phase rotations with parameters being pi    
    divided by decreasing powers of two, and then apply a Hadamard 
    gate to the nth qubit of the register reg.
    """
    for i in range(0, n):
        qc.cu1(-1*pie/float(2**(n-i)), reg[i], reg[n])
    qc.h(reg[n])
    
def add(reg_a, reg_b, circ, factor):
    """
    Add two quantum registers reg_a and reg_b, and store the result 
    in reg_a.
    """
    pie = math.pi
    n = len(reg_a)
    # Compute the Fourier transform of register a
    for i in range(0, n):
        createInputState(circ, reg_a, n - (i+1), pie)
    # Add the two numbers by evolving the Fourier transform   
    # F(ψ(reg_a))> to |F(ψ(reg_a+reg_b))>
    for i in range(0, n):
        evolveQFTState(circ, reg_a, reg_b, n - (i+1), pie, factor)
    # Compute the inverse Fourier transform of register a
    for i in range(0, n):
        inverseQFT(circ, reg_a, i, pie)

        


# p-1 operation
# Take two numbers as user input in binary form
multiplicand_in = input("Enter the prime no: P = ")
l1 = len(multiplicand_in)

#q-1 operation
multiplier_in = input("Enter the prime no: Q = .")

l2 = len(multiplier_in)

# Make sure multiplicand_in holds the larger number
if l2 > l1:
    multiplier_in, multiplicand_in = multiplicand_in, multiplier_in
    l2, l1 = l1, l2
    
accumulator = QuantumRegister(l1 + l2)
multiplicand = QuantumRegister(l1)
multiplier = QuantumRegister(l2)
d = QuantumRegister(1)
cl = ClassicalRegister(l1 + l2)
circ = QuantumCircuit(accumulator, multiplicand, multiplier, 
    d, cl, name="circ")
circ.x(d[0])

for i in range(l1):
    if multiplicand_in[i] == '1':
        circ.x(multiplicand[l1 - i - 1])
for i in range(l2):
    if multiplier_in[i] == '1':
        circ.x(multiplier[l2 - i - 1])
        
add(multiplicand, d , circ, -1)
add(multiplier, d, circ, -1)

        
multiplier_str = '1'
while(int(multiplier_str) != 0):
    add(accumulator, multiplicand, circ, 1)
    add(multiplier, d, circ, -1)
    for i in range(len(multiplier)):
        circ.measure(multiplier[i], cl[i])
    result = execute(circ,backend=Aer.get_backend('qasm_simulator'),
                shots=2).result().get_counts(circ.name)
    #print(result)
    multiplier_str = list(result.keys())[0]
    #print(multiplier_str)
    
circ.measure(accumulator, cl)
result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
            shots=2).result().get_counts(circ.name)
print("(p-1) * (q-1) =  ", result)



Enter the prime no: P = 1011
Enter the prime no: Q = .101
(p-1) * (q-1) =   {'0101000': 2}
